In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.metrics import precision_score

In [3]:
train_dataset = np.genfromtxt('train.csv', dtype=np.float32, delimiter=',')
train_labels = train_dataset[:,-1].copy()
train_labels -= 1
test_dataset = np.genfromtxt('test.csv', dtype=np.float32, delimiter=',')
test_labels = test_dataset[:,-1].copy()
test_labels -= 1
train_dataset = new_input1=np.load('new_input.npz')['new_input1']
test_dataset = new_input1=np.load('new_input.npz')['new_input2']

In [4]:
# train_labels = np.array([1 if x == 1 else 0 for x in train_labels])
# test_labels = np.array([1 if x == 1 else 0 for x in test_labels])

In [5]:
num_feature = len(train_dataset[0])
num_labels = 5
# scaler = MinMaxScaler()
# scaler.fit(train_dataset)
# train_dataset = scaler.fit_transform(train_dataset)
# test_dataset = scaler.fit_transform(test_dataset)
# l = float(len(train_dataset)) + float(len(test_dataset))
# for i in range(num_feature):
#     if len(set(train_dataset[:, i])) > 2:
#         s = float(sum(train_dataset[:, i]) + sum(test_dataset[:, i]))
#         avg = s / l
#         train_dataset[:, i] = (train_dataset[:, i] - (avg / 2)) / 2
#         test_dataset[:, i] = (test_dataset[:, i] - (avg / 2)) / 2

In [6]:
len(train_dataset)

125973

In [7]:
valid_dataset = train_dataset[-6670:, :]
valid_labels = train_labels[-6670:]
# train_dataset = train_dataset[:-6670, :]
# train_labels = train_labels[:-6670]

In [8]:
train_labels.shape

(125973,)

In [9]:
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, num_feature)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
# test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Testing set', test_dataset.shape, test_labels.shape

Training set (125973, 2000) (125973, 5)
Validation set (6670, 2000) (6670, 5)
Testing set (22544, 2000) (22544, 5)


In [10]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
def precision(predictions, labels):
    p = [np.argmax(x) for x in predictions]
    labs = [np.argmax(x) for x in labels]
    return 100 * precision_score(labs, p, average='macro')

In [40]:
batch_size = 300

graph = tf.Graph()
with graph.as_default():


    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, num_feature))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    n_hidden_1 = 3200
    n_hidden_2 = 800
    n_hidden_3 = 2000
    n_input = num_feature

    wh1 = tf.Variable(tf.truncated_normal([n_input, n_hidden_1], dtype=tf.float32))
    wh2 = tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], dtype=tf.float32))
    wh3 = tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], dtype=tf.float32))
    wo = tf.Variable(tf.truncated_normal([n_hidden_3, num_labels], dtype=tf.float32))
    b1 = tf.Variable(tf.truncated_normal([n_hidden_1], dtype=tf.float32))
    b2 = tf.Variable(tf.truncated_normal([n_hidden_2], dtype=tf.float32))
    b3 = tf.Variable(tf.truncated_normal([n_hidden_3], dtype=tf.float32))
    bo = tf.Variable(tf.truncated_normal([num_labels], dtype=tf.float32))
    
    dropout_rate = tf.placeholder(tf.float32)
    def compute_logit(x):
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, wh1), b1))
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(tf.nn.dropout(layer_1, dropout_rate), wh2), b2))
        layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(tf.nn.dropout(layer_2, dropout_rate), wh3), b3))
        return tf.matmul(tf.nn.dropout(layer_3, dropout_rate), wo) + bo

    
    logits = compute_logit(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))#\
    #+ 0.001 * (tf.nn.l2_loss(wh1) + tf.nn.l2_loss(wh2) + tf.nn.l2_loss(wo))

    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(compute_logit(tf_valid_dataset))
    test_prediction = tf.nn.softmax(compute_logit(tf_test_dataset))

In [41]:
num_steps = 20000
preds = []
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_rate: 0.7}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        if (step % 200 == 0):
            t_feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_rate: 1.0}
            print "Minibatch loss at step", step, ":", l
            bac = precision(train_prediction.eval(feed_dict = t_feed_dict), batch_labels)
            print "Minibatch precision: %.1f%%" % bac
            vac = precision(valid_prediction.eval(feed_dict = t_feed_dict), valid_labels)
            print "Validation precision: %.1f%%" % vac
    
    t_feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_rate: 1.0}
    print "Minibatch loss at step", step, ":", l
    bac = precision(train_prediction.eval(feed_dict = t_feed_dict), batch_labels)
    print "Minibatch precision: %.1f%%" % bac
    vac = precision(valid_prediction.eval(feed_dict = t_feed_dict), valid_labels) 
    print "Validation precision: %.3f%%" % vac
    preds = test_prediction.eval(feed_dict = t_feed_dict)
    print "Test precision: %.3f%%" % precision(preds, test_labels)

Initialized
Minibatch loss at step 0 : 42.9451
Minibatch precision: 11.1%
Validation precision: 17.7%
Minibatch loss at step 200 : 3.55248
Minibatch precision: 72.4%
Validation precision: 57.7%
Minibatch loss at step 400 : 1.56542
Minibatch precision: 58.7%
Validation precision: 58.5%
Minibatch loss at step 600 : 1.66146
Minibatch precision: 73.2%
Validation precision: 58.6%
Minibatch loss at step 800 : 0.806312
Minibatch precision: 72.3%
Validation precision: 78.8%
Minibatch loss at step 1000 : 1.74123
Minibatch precision: 57.9%
Validation precision: 71.3%
Minibatch loss at step 1200 : 1.32285
Minibatch precision: 88.8%
Validation precision: 74.8%
Minibatch loss at step 1400 : 0.312152
Minibatch precision: 99.5%
Validation precision: 78.7%
Minibatch loss at step 1600 : 0.792948
Minibatch precision: 79.9%
Validation precision: 72.9%
Minibatch loss at step 1800 : 0.852554
Minibatch precision: 86.5%
Validation precision: 68.5%
Minibatch loss at step 2000 : 0.676651
Minibatch precision: 9

In [42]:
preds_ = [np.argmax(x) for x in preds]

In [43]:
confusion_matrix([np.argmax(x) for x in test_labels], preds_)

array([[6053, 1325,   80,    0,    0],
       [  64, 9009,  635,    3,    0],
       [ 177,  387, 1795,   62,    0],
       [   0, 2280,   42,  232,    0],
       [   0,  393,    1,    6,    0]])

In [72]:
precision_score([np.argmax(x) for x in test_labels], preds_)

0.67293369007158865